In [1]:
from __future__ import print_function, division
from builtins import range
import numpy as np

In [2]:
#Environment
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
        
    def set(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions
        
    def set_state(self, state):
        self.i = state[0]
        self.j = state[1]
        
    def current_state(self):
        return(self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions 
    
    def move(self, action):
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action =='D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1 
        
        return self.rewards.get((self.i, self.j), 0)
        
    
    def undo_move(self):
        if action == 'U':
            self.i += 1
        elif action =='D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1 
        
        assert(self.current_state() in self.all_states())
                
            
    def game_over(self):
        return (self.i, self.j) not in self.actions
    
    
    def all_states(self):
        return set().union(self.actions.keys(), self.rewards.keys())
        #return set(self.actions.keys() + self.rewards.keys())
    

In [3]:
def standard_grid():
    # .  .  .  1
    # .  x  .  -1
    # s  .  .  .
    
    # 1, -1 are rewards for the corresponding positions
    # 'x'  not allowed
    # 's'  start state
    grid = Grid(3, 4, (2,0))
    rewards = {(0,3) : 1, (1,3) : -1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
    }
    grid.set(rewards, actions)
    return grid

In [4]:
def negative_grid(step_cost=-0.1):
    grid = standard_grid()
    grid.rewards.update({
            (0, 0): step_cost,
            (0, 1): step_cost,
            (0, 2): step_cost,
            (1, 0): step_cost,
            (1, 2): step_cost,
            (2, 0): step_cost,
            (2, 1): step_cost,
            (2, 2): step_cost,
            (2, 3): step_cost,
    })
    return grid

In [5]:
def print_values(V, grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") 
        print("")

In [6]:
def print_policy(P, grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            a = P.get((i,j), ' ')
            print("  %s  |" % a, end="")
        print("")

In [7]:
CONVERGENCE_THRESHOLD = 10e-4
GAMMA = 0.9
POSSIBLE_ACTIONS = ('U','D','L','R') 

In [10]:
if __name__ == '__main__':

    # find the value function for the given policy
    # (p(s',r|s,a)  is deterministic
    
    # negative grid gives a reward of -0.1 for every terminal state
    grid = negative_grid()

    print("Print the reward")
    print_values(grid.rewards, grid)
    
    # choose random actions
    policy = {}
    for s in grid.actions.keys():
        policy[s] =  np.random.choice(POSSIBLE_ACTIONS)
    
    print("\n\nInitial policy")
    print_policy(policy, grid)
    
    
    states = grid.all_states()
    
    # unifrom random action policy
    V = {}
    for s in states:
        if s in grid.actions:
            V[s] =  np.random.random()
        else:
            V[s] = 0

    # Repeat till policy does not change
    while True:
        
        # Policy Evaluation 
        while True:
            max_change = 0 
            for s in states:
                old_v = V[s]
            
                if s in policy:
                    a = policy[s]                
                    grid.set_state(s)
                    r = grid.move(a)
                    V[s] = r + GAMMA * V[grid.current_state()]
                    max_change = max(max_change, np.abs(old_v - V[s]))
        
            if max_change < CONVERGENCE_THRESHOLD:
                break
                
        
        # Policy improvement
        has_policy_converged = True
        for s in states:
            if s in policy:
                old_a = policy[s]
                new_a = None
                best_value = float('-inf')
                
                # look through possible actions to find the best value actions
                for a in POSSIBLE_ACTIONS:
                    grid.set_state(s)
                    r = grid.move(a)
                    v = r + GAMMA * V[grid.current_state()]
                    if v > best_value:
                        best_value = v
                        new_a = a
                
                policy[s] = new_a
                
                if old_a != new_a:
                        has_policy_converged = False
        
        if has_policy_converged:
            break
                    
                    
    print("\n\nValues")
    print_values(V, grid)
        
    print("\n\nPolicy")
    print_policy(policy, grid)
   

Print the reward
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


Initial policy
---------------------------
  L  |  U  |  L  |     |
---------------------------
  L  |     |  D  |     |
---------------------------
  U  |  D  |  R  |  U  |


Values
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|


Policy
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
